In [3]:
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
import pyforest
import requests
from bs4 import BeautifulSoup

In [2]:
#connecting to webpage and reading it into python and printing the first 500 characters of HTML
r = requests.get('https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html')
print(r.text[0:500])

<!DOCTYPE html>
<!--[if (gt IE 9)|!(IE)]> <!--><html lang="en" class="no-js page-interactive section-opinion page-theme-standard tone-opinion page-interactive-default limit-small layout-xlarge app-interactive" itemid="https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html" itemtype="http://schema.org/NewsArticle" itemscope xmlns:og="http://opengraphprotocol.org/schema/"><!--<![endif]-->
<!--[if IE 9]> <html lang="en" class="no-js ie9 lt-ie10 page-interactive section-opinion page


In [4]:
#Parsing the HTML using beautiful soup
soup = BeautifulSoup(r.text, 'html.parser')

In [5]:
#collecting the records and printing the total number of rows
results = soup.find_all('span', attrs = {'class': 'short-desc'})
len(results)

180

In [7]:
#verifying the last row matches the last tweet. 
results[-1]

<span class="short-desc"><strong>Nov. 11 </strong>“I'd rather have him  – you know, work with him on the Ukraine than standing and arguing about whether or not  – because that whole thing was set up by the Democrats.” <span class="short-truth"><a href="https://www.nytimes.com/interactive/2017/12/10/us/politics/trump-and-russia.html" target="_blank">(There is no evidence that Democrats "set up" Russian interference in the election.)</a></span></span>

In [14]:
#building the dataset 
#task1: extracting the date
first_result = results[0]
#displaying full output
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
first_result.find('strong')
first_result.find('strong').text
first_result.find('strong').text[0:-1]
first_result.find('strong').text[0:-1] + ', 2017'

<strong>Jan. 21 </strong>

'Jan. 21\xa0'

'Jan. 21'

'Jan. 21, 2017'

In [22]:
#task 2 extracting the lie (tweet)
first_result
first_result.contents
first_result.contents[1]
first_result.contents[1][1:-2]

<span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Iraq. I didn't want to go into Iraq.” <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span></span>

[<strong>Jan. 21 </strong>,
 "“I wasn't a fan of Iraq. I didn't want to go into Iraq.” ",
 <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span>]

"“I wasn't a fan of Iraq. I didn't want to go into Iraq.” "

"I wasn't a fan of Iraq. I didn't want to go into Iraq."

In [26]:
#tast 3 extracing the explanation 
first_result
first_result.contents
first_result.contents[2]
first_result.find('a')
first_result.find('a').text
first_result.find('a').text[1:-1]

<span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Iraq. I didn't want to go into Iraq.” <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span></span>

[<strong>Jan. 21 </strong>,
 "“I wasn't a fan of Iraq. I didn't want to go into Iraq.” ",
 <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span>]

<span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span>

<a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a>

'(He was for an invasion before he was against it.)'

'He was for an invasion before he was against it.'

In [28]:
#task 4 extracting the URL 
first_result.find('a')
first_result.find('a')['href']

<a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a>

'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the'

In [29]:
#task 5 building dataset using loop function 

records = []
for result in results: 
    date = result.find('strong').text[0:-1] + ', 2017'
    lie = result.contents[1][1:-2]
    explanation = first_result.find('a').text[1:-1]
    url = first_result.find('a')['href']
    records.append((date, lie, explanation, url))

In [31]:
records[0:3]
len(records)

[('Jan. 21, 2017',
  "I wasn't a fan of Iraq. I didn't want to go into Iraq.",
  'He was for an invasion before he was against it.',
  'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the'),
 ('Jan. 21, 2017',
  'A reporter for Time magazine — and I have been on their cover 14 or 15 times. I think we have the all-time record in the history of Time magazine.',
  'He was for an invasion before he was against it.',
  'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the'),
 ('Jan. 23, 2017',
  'Between 3 million and 5 million illegal votes caused me to lose the popular vote.',
  'He was for an invasion before he was against it.',
  'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the')]

180

In [36]:
#task 6 applying tabular structure using pandas
df = pd.DataFrame(records,columns = ['date','lie', 'explanation', 'url'] )
df.head()
df.dtypes
df['date'] = pd.to_datetime(df['date'])
df.dtypes
df.index = df.index+1
df.head()

<IPython.core.display.Javascript object>

,date,lie,explanation,url
0,"Jan. 21, 2017",I wasn't a fan of Iraq. I didn't want to go in...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
1,"Jan. 21, 2017",A reporter for Time magazine — and I have been...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
2,"Jan. 23, 2017",Between 3 million and 5 million illegal votes ...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
3,"Jan. 25, 2017","Now, the audience was the biggest ever. But th...",He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
4,"Jan. 25, 2017",Take a look at the Pew reports (which show vot...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...


date           object
lie            object
explanation    object
url            object
dtype: object

<IPython.core.display.Javascript object>

date           datetime64[ns]
lie                    object
explanation            object
url                    object
dtype: object

,date,lie,explanation,url
1,2017-01-21,I wasn't a fan of Iraq. I didn't want to go in...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
2,2017-01-21,A reporter for Time magazine — and I have been...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
3,2017-01-23,Between 3 million and 5 million illegal votes ...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
4,2017-01-25,"Now, the audience was the biggest ever. But th...",He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
5,2017-01-25,Take a look at the Pew reports (which show vot...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
